In [1]:
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
_pipeline_root = 'pipeline/'
context = InteractiveContext(pipeline_root=_pipeline_root)

2023-04-06 15:25:27.001792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 15:25:27.106491: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-06 15:25:27.106511: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-06 15:25:27.667837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, SelectKBest, SelectFromModel, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
# from sklearn.preprocessing import SandardScaler, MinMaxScaler

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Test2/data2/train.csv')
print(df.dtypes)
df.describe(include='all')

In [ ]:
df.head()

In [ ]:
df.drop('total_children',axis=1,inplace=True)

In [ ]:
cor = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(cor,annot=True,cmap=plt.cm.PuBu)

In [ ]:
cor_target = abs(cor['cost'])
relevant_features = cor_target[cor_target>=0.00]
names = [index for index,value in relevant_features.iteritems()]
# names.remove('cost')
names

In [ ]:
plt.figure(figsize=(20,20))
new_corr = df[names].corr()
sns.heatmap(new_corr,annot=True,cmap=plt.cm.Blues)

In [ ]:
new_names = [x for x in names if x not in ['gross_weight','units_per_case','recyclable_package']]
new_names

In [ ]:
df[new_names].head()

In [ ]:
# df.drop('prepared_food',axis=0,inplace=True)
df[new_names].to_csv('data/train_features.csv', index=False)

In [31]:
import pandas as pd
df2 = pd.read_csv('data/train_features.csv')
df2.head()

,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,low_fat,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,8.61,3.0,2.0,2.0,2.0,0.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.09
1,5.00,2.0,4.0,0.0,3.0,0.0,28206.0,1.0,0.0,0.0,0.0,0.0,121.80
2,14.08,4.0,0.0,0.0,3.0,0.0,21215.0,1.0,0.0,0.0,0.0,0.0,83.51
3,4.02,3.0,5.0,0.0,0.0,1.0,21215.0,1.0,0.0,0.0,0.0,0.0,66.78
4,2.13,3.0,5.0,0.0,3.0,1.0,27694.0,1.0,1.0,1.0,1.0,1.0,111.51


In [3]:
from tfx.proto import example_gen_pb2
import os
from tfx.components import CsvExampleGen


output= example_gen_pb2.Output(
split_config=example_gen_pb2.SplitConfig(splits=[
    example_gen_pb2.SplitConfig.Split(name='train',hash_buckets=8),
    example_gen_pb2.SplitConfig.Split(name='eval',hash_buckets=2)
    ])   
)

examples_gen = CsvExampleGen(input_base='data',output_config=output)
context.run(examples_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [4]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=examples_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [5]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [6]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'avg_cars_at home(approx).1',FLOAT,required,,-
'coffee_bar',FLOAT,required,,-
'cost',FLOAT,required,,-
'florist',FLOAT,required,,-
'low_fat',FLOAT,required,,-
'num_children_at_home',FLOAT,required,,-
'prepared_food',FLOAT,required,,-
'salad_bar',FLOAT,required,,-
'store_sales(in millions)',FLOAT,required,,-


In [7]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [8]:
shops_constants_file = 'shops_constants.py'

In [ ]:
%%writefile {shops_constants_file}

NORMALIZE_TO_0_1 = {'store_sales(in millions)':None,'store_sqft':None}
FLOAT_TO_INT = {'avg_cars_at home(approx).1':None,'coffee_bar':None,'florist':None,'salad_bar':None,'unit_sales(in millions)':None,'video_store':None,'total_children':None,'num_children_at_home':None,'low_fat':None,'prepared_food':None}
LABEL_CHANGE = {'cost':None}

def transformed_name(key):
    return key + '_xf'


In [9]:
shops_transform_file = 'shops_transform.py'

In [33]:
%%writefile {shops_transform_file}

import tensorflow as tf
import tensorflow_transform as tft

import shops_constants

transformed_name = shops_constants.transformed_name
NORMALIZE_TO_0_1 = shops_constants.NORMALIZE_TO_0_1
FLOAT_TO_INT = shops_constants.FLOAT_TO_INT
LABEL_CHANGE = shops_constants.LABEL_CHANGE

def preprocessing_fn(inputs):
    outputs = {}

    for key in FLOAT_TO_INT.keys():
        key_data3 = inputs[key]
        changed_data3 = tf.cast(key_data3,dtype=tf.int64)
        outputs[transformed_name(key)] = changed_data3

    for key in NORMALIZE_TO_0_1.keys():
        key_data2 = inputs[key]
        scaled_data2 = tft.scale_to_0_1(key_data2)
        outputs[transformed_name(key)] = scaled_data2
    
    for key in LABEL_CHANGE.keys():
        key_data4 = inputs[key]
        scaled_data4 = key_data4#tft.scale_to_0_1(key_data4)
        outputs[transformed_name(key)] = scaled_data4


    return outputs

Overwriting shops_transform.py


In [34]:
import tfx
transform = tfx.components.Transform(
    examples = examples_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file=os.path.abspath(shops_transform_file)
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying inference.py -> build/lib
copying shops_trainer.py -> build/lib
copying shops_constants.py -> build/lib
copying shops_transform.py -> build/lib
installing to /tmp/tmps7_eh6_9
running install
running install_lib
copying build/lib/shops_transform.py -> /tmp/tmps7_eh6_9
copying build/lib/shops_constants.py -> /tmp/tmps7_eh6_9
copying build/lib/shops_trainer.py -> /tmp/tmps7_eh6_9
copying build/lib/inference.py -> /tmp/tmps7_eh6_9
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.e

/home/thakur/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


creating /tmp/tmps7_eh6_9/tfx_user_code_Transform-0.0+07a7116ca9c9623305d0409f14f6c38aa867568fee5b0028bbb4c9ada12daee1.dist-info/WHEEL
creating '/tmp/tmpgitz59ad/tfx_user_code_Transform-0.0+07a7116ca9c9623305d0409f14f6c38aa867568fee5b0028bbb4c9ada12daee1-py3-none-any.whl' and adding '/tmp/tmps7_eh6_9' to it
adding 'inference.py'
adding 'shops_constants.py'
adding 'shops_trainer.py'
adding 'shops_transform.py'
adding 'tfx_user_code_Transform-0.0+07a7116ca9c9623305d0409f14f6c38aa867568fee5b0028bbb4c9ada12daee1.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+07a7116ca9c9623305d0409f14f6c38aa867568fee5b0028bbb4c9ada12daee1.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+07a7116ca9c9623305d0409f14f6c38aa867568fee5b0028bbb4c9ada12daee1.dist-info/top_level.txt'
adding 'tfx_user_code_Transform-0.0+07a7116ca9c9623305d0409f14f6c38aa867568fee5b0028bbb4c9ada12daee1.dist-info/RECORD'
removing /tmp/tmps7_eh6_9
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+07a7116ca9c9623305d

INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/15/.temp_path/tftransform_tmp/34ea01d3ce58407181fbbcb2a17b78b8/assets


INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/15/.temp_path/tftransform_tmp/34ea01d3ce58407181fbbcb2a17b78b8/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 15
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [35]:
trainer_module_file = 'shops_trainer.py'

In [40]:
%%writefile {trainer_module_file}

import tensorflow as tf
# import tensorflow_decision_forests as tfdf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from absl import logging
from tensorflow.keras import layers, Model, optimizers, losses, metrics
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from typing import List, Text
import shops_constants
import os

transformed_name = shops_constants.transformed_name
NORMALIZE_TO_0_1 = shops_constants.NORMALIZE_TO_0_1
FLOAT_TO_INT = shops_constants.FLOAT_TO_INT

def model_fn():
    inputs = {}
    for key,item in FLOAT_TO_INT.items():
        inputs[transformed_name(key)] = tf.keras.Input(shape=(1,),name=transformed_name(key))

    for key,item in NORMALIZE_TO_0_1.items():
        inputs[transformed_name(key)] = tf.keras.Input(shape=(1,),name=transformed_name(key))
    # inputs['dental caries_xf'] = tf.keras.Input(shape=(1,),name=transformed_name('dental caries'))
    output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
    # output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(128, activation='relu')(output)
    # output = tf.keras.layers.Dropout(0.05)(output)
    # output = tf.keras.layers.BatchNormalization()(output)
    # output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dense(16, activation='relu')(output)
    output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
#     output = tf.keras.layers.Dense(16, activation='relu')(output)
    output = tf.keras.layers.Dense(1)(output)
    model = tf.keras.models.Model(inputs,output)

    # Compile model
    model.compile(optimizer='adam',#tf.keras.optimizers.Adam(learning_rate=0.1),
                loss='mse',#mae
                metrics=['mean_squared_logarithmic_error'])

    return model





LABEL_KEY = 'cost_xf'

def _gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames,compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, batch_size=2000):
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=LABEL_KEY
    )
    return dataset

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """
    Returns a function that parses a serialized tf.Example and applies
    the transformations during inference.
    Args:
        model: The model that we are serving.
        tf_transform_output: The transformation output that we want to 
            include with the model.
    """
    
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        
        required_feature_spec = {
            k: v for k, v in feature_spec.items() if k != 'cost'
        }

        parsed_features = tf.io.parse_example(
            serialized_tf_examples,
            required_feature_spec
        )

        transformed_features = model.tft_layer(parsed_features)
        output = model(transformed_features)

        return output

    return serve_tf_examples_fn

def run_fn(fn_args):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    train_dataset = input_fn(fn_args.train_files,tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files,tf_transform_output)

    model = model_fn()

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir),'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,update_freq='batch')

    model.fit(
        x=train_dataset,
        steps_per_epoch = fn_args.train_steps,
        validation_data = eval_dataset,
        validation_steps = fn_args.eval_steps,
        epochs=200,
        callbacks=[tensorboard_callback]
    )
    

    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, 
                                 tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples')),
    }
    model.save(fn_args.serving_model_dir,save_format='tf',signatures=signatures,overwrite=True)#
    # print("Model saved to:", fn_args.serving_model_dir)


Overwriting shops_trainer.py


In [41]:
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2
import tensorflow_transform as tft

TRAINING_STEPS = 1000
EVALUTION_STEPS = 400

trainer = Trainer(
    module_file=trainer_module_file,
    custom_executor_spec = executor_spec.ExecutorClassSpec(GenericExecutor),
    transform_graph = transform.outputs['transform_graph'],
    transformed_examples = transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args = trainer_pb2.EvalArgs(num_steps=EVALUTION_STEPS)
)

context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying inference.py -> build/lib
copying shops_trainer.py -> build/lib
copying shops_constants.py -> build/lib
copying shops_transform.py -> build/lib
installing to /tmp/tmp9ah594n9
running install
running install_lib
copying build/lib/shops_transform.py -> /tmp/tmp9ah594n9
copying build/lib/shops_constants.py -> /tmp/tmp9ah594n9
copying build/lib/shops_trainer.py -> /tmp/tmp9ah594n9
copying build/lib/inference.py -> /tmp/tmp9ah594n9
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCE

/home/thakur/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


creating /tmp/tmp9ah594n9/tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4.dist-info/WHEEL
creating '/tmp/tmpwblcj1uv/tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4-py3-none-any.whl' and adding '/tmp/tmp9ah594n9' to it
adding 'inference.py'
adding 'shops_constants.py'
adding 'shops_trainer.py'
adding 'shops_transform.py'
adding 'tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4.dist-info/METADATA'
adding 'tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4.dist-info/WHEEL'
adding 'tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4.dist-info/top_level.txt'
adding 'tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4.dist-info/RECORD'
removing /tmp/tmp9ah594n9


Processing ./pipeline/_wheels/tfx_user_code_Trainer-0.0+c3dbdb72768ae9b2af5437aacfac792ad1f6371b7090b6ba3e740ce3d6a336b4-py3-none-any.whl
Epoch 1/200
1000/1000 [==============================] - 13s 13ms/step - loss: 1654.8634 - mean_squared_logarithmic_error: 0.6364 - val_loss: 882.1711 - val_mean_squared_logarithmic_error: 0.1005
Epoch 2/200
1000/1000 [==============================] - 13s 13ms/step - loss: 872.5045 - mean_squared_logarithmic_error: 0.0996 - val_loss: 866.5239 - val_mean_squared_logarithmic_error: 0.0989
Epoch 3/200
1000/1000 [==============================] - 12s 12ms/step - loss: 865.6513 - mean_squared_logarithmic_error: 0.0988 - val_loss: 861.9302 - val_mean_squared_logarithmic_error: 0.0979
Epoch 4/200
1000/1000 [==============================] - 13s 13ms/step - loss: 861.6853 - mean_squared_logarithmic_error: 0.0984 - val_loss: 857.8563 - val_mean_squared_logarithmic_error: 0.0982
Epoch 5/200
1000/1000 [==============================] - 12s 12ms/step - loss: 85

1000/1000 [==============================] - 9s 9ms/step - loss: 816.2374 - mean_squared_logarithmic_error: 0.0930 - val_loss: 822.1985 - val_mean_squared_logarithmic_error: 0.0953
Epoch 42/200
1000/1000 [==============================] - 11s 11ms/step - loss: 815.7649 - mean_squared_logarithmic_error: 0.0929 - val_loss: 817.8039 - val_mean_squared_logarithmic_error: 0.0932
Epoch 43/200
1000/1000 [==============================] - 8s 8ms/step - loss: 815.5685 - mean_squared_logarithmic_error: 0.0929 - val_loss: 818.8893 - val_mean_squared_logarithmic_error: 0.0928
Epoch 44/200
1000/1000 [==============================] - 9s 9ms/step - loss: 814.9843 - mean_squared_logarithmic_error: 0.0928 - val_loss: 820.8007 - val_mean_squared_logarithmic_error: 0.0918
Epoch 45/200
1000/1000 [==============================] - 10s 10ms/step - loss: 814.2456 - mean_squared_logarithmic_error: 0.0927 - val_loss: 816.8704 - val_mean_squared_logarithmic_error: 0.0931
Epoch 46/200
1000/1000 [===============

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipeline/Trainer/model/18/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline/Trainer/model/18/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 18
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [16]:
model_path = trainer.outputs['model'].get()[0].uri
model_path

'pipeline/Trainer/model/6'

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {model_path}

In [81]:
import tensorflow_model_analysis as tfma
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=model_path,
    tags=[tf.saved_model.SERVING]
)

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='cost')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
    tfma.MetricsSpec(metrics=[
    # tfma.MetricConfig(class_name='BinaryAccuracy'),
    tfma.MetricConfig(class_name='ExampleCount'),
    # tfma.MetricConfig(class_name='FalsePositives'),
    # tfma.MetricConfig(class_name='TruePositives'),
    # tfma.MetricConfig(class_name='FalseNegatives'),
    # tfma.MetricConfig(class_name='TrueNegatives'),
    # tfma.MetricConfig(class_name='Precision'),
    # tfma.MetricConfig(class_name='Recall')
    ])
    ]
)

from tfx.components import Evaluator

evaluator = Evaluator(
     examples=examples_gen.outputs['examples'],
     model=trainer.outputs['model'],
     eval_config=eval_config)

context.run(evaluator) 

ExecutionResult(
    component_id: Evaluator
    execution_id: 13
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [117]:
eval_results = evaluator.outputs['evaluation'].get()[0].uri
tfma_results = tfma.load_eval_result(eval_results)

In [115]:
tfma.view.render_slicing_metrics(tfma_results)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [121]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0


In [120]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

missing_thresholds: true



In [43]:
# eval_results = evaluator.outputs['evaluation'].get()[0].uri
# tfma_results = tfma.load_eval_result(eval_results)

# tfma.view.render_slicing_metrics(tfma_results)

from tfx.components import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = 'Serve/'

pusher = Pusher(
    model = trainer.outputs['model'],
#     model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
    base_directory=_serving_model_dir
    )
))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 20
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [ ]:
import pandas as pd
train_msle = pd.read_csv('logs_train.csv')
eval_msle = pd.read_csv('logs_validation.csv')
train_msle_fs = pd.read_csv('logs_train_fs.csv')
eval_msle_fs = pd.read_csv('logs_validation_fs.csv')

In [ ]:
%matplotlib qt

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(train_msle['Step'],train_msle['Value'])
plt.plot(train_msle['Step'],train_msle_fs['Value'])

In [ ]:
plt.plot(train_msle['Step'],eval_msle['Value'],'k')
plt.plot(train_msle['Step'],eval_msle_fs['Value'],'b')

In [23]:
def serving_input_receiver_fn():
    input_feature = tf.compat.v1.placeholder(dtype=tf.string,shape=[None,1],name='input')
    fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        features={'input_feature':input_feature}
    )
    return fn

In [66]:
loaded_model = tf.saved_model.load('pipeline/Trainer/model/6/6/')

In [110]:
import requests
import json

# The URL of the prediction endpoint
url = 'http://localhost:8501/v1/models/6:predict'

# The input data for prediction
input_data = {
    'instances': [
        {'store_sales(in millions)':8.61,'unit_sales(in millions)':3.0,	'total_children':2.0,'num_children_at_home':2.0,'avg_cars_at home(approx).1':2.0,'low_fat':0.0,	'store_sqft':36509.0,'coffee_bar':0,'video_store':0,'salad_bar':0,'prepared_food':0,'florist':0}
    ]
}
# 'store_sales(in millions)':8.61,'unit_sales(in millions)':3.0,	'total_children':2.0,'num_children_at_home':2.0,'avg_cars_at home(approx).1':2.0,'low_fat':0.0,	'store_sqft':36509.0,'coffee_bar':0,'video_store':0,'salad_bar':0,'prepared_food':0,'florist':0
# Convert the input data to a JSON string
input_data_json = json.dumps(input_data)

# Set the content type of the request
headers = {'content-type': 'application/json'}

# Send the prediction request
response = requests.post(url, data=input_data_json, headers=headers)

# Get the prediction results
prediction_results = json.loads(response.text)

In [111]:
prediction_results

{'error': "Failed to process element: 0 key: store_sales(in millions) of 'instances' list. Error: INVALID_ARGUMENT: JSON object: does not have named input: store_sales(in millions)"}

In [67]:
print(loaded_model.signatures)

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, examples) at 0x7FBC61747340>})


In [103]:
import requests
import json
import base64

# Define the URL of the Docker container
url = 'http://localhost:8501/v1/models/6:predict'
input_data = {"inputs": [1.0, 2.0, 3.0, 4.0]}

# convert the dictionary to a JSON string
json_data = json.dumps(input_data)

# encode the JSON string as base64
encoded_data = base64.b64encode(json_data.encode('utf-8')).decode('utf-8')

# create the prediction request
data = {
    "instances": [
        {"b64": encoded_data}
    ]
}

# send the prediction request
response = requests.post(url, json=data)

In [104]:
prediction_results = json.loads(response.text)

In [105]:
prediction_results

{'error': 'Could not parse example input, value: \'{"inputs": [1.0, 2.0, 3.0, 4.0]}\'\n\t [[{{function_node __inference_serve_tf_examples_fn_138243}}{{node ParseExample/ParseExampleV2}}]]'}

In [114]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

OSError: SavedModel file does not exist at: pipeline/Pusher/pushed_model/10/{saved_model.pbtxt|saved_model.pb}